In [5]:
%run SentimentDictionary.ipynb

In [43]:
import numpy as np
import pandas as pd
import re
import xlrd
import nltk
from sklearn.metrics import accuracy_score

In [40]:
#import label data
data_path= 'SentimentCleaning.xlsx'
train=pd.read_excel(data_path, sep = ';')

In [41]:
train.head()

,Content,remove,tokenize_sents,Mean
0,en kvinna föll och ådrog sig en skada i ena ax...,föll ådrog skada ena axeln fortfarande stora s...,"['föll', 'ådrog', 'skada', 'ena', 'axeln', 'fo...",-1
1,en kvinna hade genomgått en operation och vård...,genomgått vårdades tillsammans andra patienter...,"['genomgått', 'vårdades', 'tillsammans', 'andr...",-1
2,en man med symtom från mag tarmkanalen sökte a...,symtom akutmottagning sjukhuset gånger rätt di...,"['symtom', 'akutmottagning', 'sjukhuset', 'gån...",-1
3,en kvinna med bröstcancer har genomgått strålb...,genomgått ena bröstet behandlingen försökt kom...,"['genomgått', 'ena', 'bröstet', 'behandlingen'...",-1
4,en kvinna hade problem med trötthet och stort ...,problem stort genomgick undersökningar svar vi...,"['problem', 'stort', 'genomgick', 'undersöknin...",0


In [10]:
afinn = Afinn(language='sv')
afinn_scores = [afinn.score(text) for text in train.remove]
train['Afinn'] = afinn_scores
train['Afinn'] = train['Afinn'].astype('int').copy()

,Content,remove,Afinn
0,en kvinna föll och ådrog sig en skada i ena ax...,föll ådrog skada ena axeln fortfarande stora s...,-6
1,en kvinna hade genomgått en operation och vård...,genomgått vårdades tillsammans andra patienter...,2
2,en man med symtom från mag tarmkanalen sökte a...,symtom akutmottagning sjukhuset gånger rätt di...,0
3,en kvinna med bröstcancer har genomgått strålb...,genomgått ena bröstet behandlingen försökt kom...,-2
4,en kvinna hade problem med trötthet och stort ...,problem stort genomgick undersökningar svar vi...,-5


In [35]:
saldo = SenSaldo(language='se')
saldo_scores = [saldo.score(text) for text in train.remove]
#saldo_scores = [saldo.score(text) for text in train.Content]
train['SenSaldo'] = saldo_scores
train['SenSaldo'] = train['SenSaldo'].astype('int').copy()

In [16]:
#Combine the Afinn ratings for sentiment analysis(3 levels:'-2','-1','0')
conditions = [
    (train['Afinn']>=-4),
    (train['Afinn']<-4) & (train['Afinn']>-7),
    (train['Afinn']<-6)]
choices = [0,-1,-2]
train['afinn-label'] = np.select(conditions, choices, default='black')
train['afinn-label'] = train['afinn-label'].astype('int64')

In [36]:
#Combine the Sensaldo ratings for sentiment analysis(3 levels:'-2','-1','0')
conditions=[
    (train['SenSaldo']>=-2),
    (train['SenSaldo']<-2) & (train['SenSaldo']>-8),
    (train['SenSaldo']<-7)
]
choices = [0,-1,-2]
train['saldo-label']=np.select(conditions, choices, default='black')
train['saldo-label']=train['saldo-label'].astype('int64')

In [39]:
#Afinn dictionary Accuracy
accuracy_score(train['Mean'],train['afinn-label'])

0.6101083032490975

In [38]:
#Sensaldo dictionary Accuracy
accuracy_score(train['Mean'],train['saldo-label'])

0.6353790613718412

,remove,Mean,afinn-label,saldo-label
0,föll ådrog skada ena axeln fortfarande stora s...,-1,-1,-1
1,genomgått vårdades tillsammans andra patienter...,-1,0,-1
2,symtom akutmottagning sjukhuset gånger rätt di...,-1,0,0
3,genomgått ena bröstet behandlingen försökt kom...,-1,0,0
4,problem stort genomgick undersökningar svar vi...,0,-1,0
